#### Описание данных 

- *date – дата совершения события*

- *event - событие*\
 app_install – установка приложения\
 app_start – открыть приложения\
 register – зарегистрироваться \
 search – перейти на страницу поиска товаров (каталог)\
 open_item – открыть товар\
 choose_item – отправить товар в корзину\
 tap_basket – перейти в корзину\
 purchase – подтверждение покупки\

- *gender – пол пользователя*

- *os_name – платформа пользователя*

- *city – город пользователя*

- *device_id – идентификатор устройства пользователя*

- *urm_source – канал, с которого пришел пользователь*\
 yandex-direct – Яндекс директ\
 google_ads – реклама в Google\
 vk_ads – реклама в ВК\
 instagram_ads – реклама в instagram\
 facebook_ads – реклама в facebook\
 referal – акция «приведи друга»\
 Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы \ 
- *purchase_sum – стоимость покупки (при совершении события ‘purchase’)*

#### 1. Предложить какие метрики из предложенных смогут определить насколько хорошо маркетологи справляются со своей задачей привлечения пользователей 

* DAU (количество уникальных пользователей, которые зашли в приложение в течение суток) не подходит, так как в эту метрику могут входить все сегменты пользователей (как пользователи, которых маргетологи привлекли вчера-сегодня, так и пользователи, которые давно пользуются продуктом). Метрика общая и включает в себя усилия не только маркетологов, но и усилия продакт менеджеров.
* **CAC (стоимость привлечения клиента)** подходит, так как эта метрика помогает нам понять насколько дорого или дешево маркетологи привлекли нам клиентов. То есть, если один платящий пользователь (совершивший целевое действие) стоит недорого относительно индустрии или относительно, например, работы периодов работы других маркетологов, то это показатель хорошей работы маркетолога, а при отрицательных значениях - плохой работы маркетолога.
* Количество сессий на пользователя (помогает понять как часто пользователи возвращаются и используют сайт) не подходит также как и DAU в нашем случае ни о чем не говорит. Это могут быть количество переходов в продукт, количество лояльных пользователей, пользователей "старичков". 
* Retention n-ого дня - не подходит, так как метрика исключительно продуктовая.
* **Количество инсталлов** подходит, так как это именно количество новых пользователей, которые пришли в продукт. Если маркетологи находят много целевых пользователей и они конвертируются в установку, а дальше по воронке доходят до покупки, то это показатель хорошей работы маркетолога.

#### 2. Предложить какая метрика в динамике отражает работу продуктовой команды и продакт менеджера

Денежные метрики нам не подходят, потому что денежные метрики - это хорошие прокси метрики. То есть, выручка уже исключается из вариантов.

* DAU, MAU метрики включают в себя общие усилия как продутовой команды, так и маркетологов, поэтому нам не подходит.
* CR из установки в открытие приложения не подходит, потому что эта метрика показывает нам насколько действительно целевой трафик пригнал маркетинг. Если конверсия низкая, то маркетологи привлекли нецелевой трафик.
* CR из открытия приложения в покупку не подходит, так как, например, пользователи могут дать продукту шанс и купить в нём что-то, но понравилось ли им приложение мы увидим только если они вернутся в приложение.
* **CR в повторную покупку в течение месяца** подходит, так как определяет возвращаются ли пользователи в продукт.

#### 3. Посчитать MAU февраля

In [1]:
import pandas as pd

In [35]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-b-dzhumaeva-37/product_analytics/KC_case_data .csv', 
                parse_dates=[0])

In [3]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


Посмотрим на event

In [4]:
df.event.value_counts()

app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: event, dtype: int64

Нас интересует событие app_start

In [22]:
df['month'] = df.date.dt.strftime('%Y-%m')

In [6]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
month                   object
dtype: object

In [7]:
df.query("month == '2020-02' & event == 'app_start'").device_id.nunique()

75032

#### 4. Посчитать количество установок в январе

In [8]:
df.query("month == '2020-01' & event == 'app_install'").device_id.nunique()

80297

#### 5. Присвоить пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Определить для какой когорты конверсия была наибольшей.

Примечание: считаю пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [9]:
installs = df[df['event']=='app_install'][['date', 'device_id']]

In [11]:
installs = installs.drop_duplicates('device_id')

При удалении дупликатов по дефолту всегда остается первое значение, поэтому у нас остаются только даты установки для каждого id, которая является его когортой

In [12]:
installs = installs.rename(columns = {'date':'installs_date'})

In [13]:
df = df.merge(installs, how='left', on='device_id')

In [14]:
purchases = df[df['event'] == 'purchase'][['date','device_id']]
purchases = purchases.sort_values('date').drop_duplicates('device_id')
purchases = purchases.rename(columns={'date':'first_purchase_date'})

In [15]:
df = df.merge(purchases, how='left', on='device_id')

In [16]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,installs_date,first_purchase_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01,2020-01-01,NaT
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01,2020-01-01,NaT


Сохраняю новый датасет для последующей обработки в Tableau

In [17]:
df.to_csv('KC_result_csv', index=False)

Также можно обойтись инструментами python, но подход будет чуть другой:\
снова сохраняю изначальный датасет

In [23]:
df = df.drop('month', axis=1)

In [25]:
installs = df.query('event=="app_install"').rename(columns={'date':'install_date'})
installs.head()

,install_date,event,purchase_sum,os_name,device_id,gender,city,utm_source
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal


In [26]:
purchases=df.query('event=="purchase"').rename(columns={'date':'purchase_date'})\
            .sort_values('purchase_date').drop_duplicates('device_id',keep='first')
purchases.head()

,purchase_date,event,purchase_sum,os_name,device_id,gender,city,utm_source
7691,2020-01-01,purchase,265.5,android,6081155,female,Moscow,-
8289,2020-01-01,purchase,292.0,ios,1955306,female,Moscow,yandex-direct
8290,2020-01-01,purchase,664.0,ios,20176327,female,Moscow,facebook_ads
8291,2020-01-01,purchase,337.0,android,28603960,male,Moscow,-
8292,2020-01-01,purchase,502.0,ios,14028316,female,Moscow,referal


In [27]:
cr_df=installs.merge(purchases, how='left', on=['device_id'])
cr_df.head()

,install_date,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,purchase_date,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10,purchase,311.0,ios,male,Moscow,-
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01,purchase,479.5,android,male,Saint-Petersburg,-
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
cr_df['date_diff']=cr_df.purchase_date-cr_df.install_date
cr_df['date_diff']=cr_df.date_diff.dt.days

In [29]:
cr_df.head()

,install_date,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,purchase_date,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y,date_diff
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10,purchase,311.0,ios,male,Moscow,-,9.0
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01,purchase,479.5,android,male,Saint-Petersburg,-,0.0
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
cr_df.date_diff=cr_df.date_diff.where(cr_df.date_diff<8)

In [31]:
cr_df.head()

,install_date,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,purchase_date,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y,date_diff
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10,purchase,311.0,ios,male,Moscow,-,NaN
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01,purchase,479.5,android,male,Saint-Petersburg,-,0.0
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
cr=cr_df.groupby('install_date',as_index=False).agg({'event_x':'count','date_diff':'count'})
cr=cr.rename(columns={'event_x':'installs','date_diff':'purchases'})


cr.head()

,install_date,installs,purchases
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587


In [34]:
cr['cr']=round(cr.purchases/cr.installs*100, 1)
cr.sort_values('cr',ascending=False).head()

,install_date,installs,purchases,cr
0,2020-01-01,3579,1408,39.3
8,2020-01-09,1424,558,39.2
14,2020-01-15,4310,1650,38.3
13,2020-01-14,5173,1973,38.1
1,2020-01-02,3144,1186,37.7


Таким образом, конверсия была наибольшей для когорты '2020-01-01' и она составляет 39.3%

#### 6. Выяснить с какого платного маркетингового канала пришло больше всего новых пользователей

Первое, что нужно учесть - это то, что мы оставляем в анализе только платные каналы, то есть мы исключаем органический трафик, прямые заходы.
Второе, мы учитываем только новых пользователей, которые установили приложение.

In [37]:
df.utm_source.value_counts() #посмотрим какие у нас каналы

-                610458
vk_ads           418456
yandex-direct    390232
instagram_ads    382891
google_ads       374257
facebook_ads     336953
referal          234721
Name: utm_source, dtype: int64

In [39]:
df.query("event=='app_install'").groupby('utm_source').agg({'device_id':'nunique'}).sort_values('device_id', ascending=False)

,device_id
utm_source,
-,32460
yandex-direct,29368
google_ads,26286
vk_ads,23189
instagram_ads,20096
facebook_ads,13916
referal,9282


Мы учитываем только платные каналы, поэтому больше всего новых пользователей у нас пришло с рекламы в яндекс-директе

#### 7. Проанализировать на каком этапе воронки отваливается бОльшая часть клиентов. Посмотреть отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. Выяснить на каком шаге отваливается больше всего зарегистрированных пользователей.

Я буду анализировать только зарегистрированных пользователей, потому что очевидно, что незарегистрированный пользователей, как бывает чаще всего, отваливается именно на этапе регистрации

In [40]:
first_registration = df.query("event=='register'").sort_values('date').drop_duplicates('device_id')

In [41]:
df['first_registration'] = df['device_id'].map(first_registration.set_index('device_id')['date'])

In [42]:
already_reg = df[df['first_registration'] < df['date']]

In [43]:
already_reg.groupby('event')['device_id'].nunique()

event
app_start      40991
choose_item    37926
purchase       24880
search         40482
tap_basket     34517
Name: device_id, dtype: int64

In [45]:
print('Конверсия в поиск из открытия', round((40482/40991)*100, 2))

Конверсия в поиск из открытия 98.76


In [47]:
print('Конверсия в добавление товара из поиска', round((37926/40482)*100, 2))

Конверсия в добавление товара из поиска 93.69


In [48]:
print('Конверсия в переход в корзину из добавления товара', round((34517/37926)*100, 2))

Конверсия в переход в корзину из добавления товара 91.01


In [49]:
print('Конверсия в покупку из перехода в корзину', round((24880/34517)*100, 2))

Конверсия в покупку из перехода в корзину 72.08


Конверсии довольно высокие для зарегистрированных пользователей, но в покупку из перехода в корзине ожидаемо ниже остальных. Если бы передо мной стояла задача проанализировать почему эта конверсия такая низкая, можно было бы первым делом исследовать саму воронку, посмотреть как работает продукт, то есть какие шаги после того, как пользователь перешел в корзину, нужно совершить до момента покупки, посмотреть конверсию в более детальном разрезе и понять на каком именно этапе (при выборе карты, при вводе каких-то платежных данных, либо когда появляется информация про доставку) отваливается наш пользователь. 

Таким образом, после перехода в корзину, у нас отваливается больше всего пользователей

#### 8. Выяснить с каких каналов приходят пользователи, показывающие самую низкую конверсия в первую покупку.

In [51]:
df_1 = df.groupby(['utm_source', 'event'], as_index=False)\
         .agg({'device_id': 'nunique'})\
         .pivot(index = 'utm_source', columns = 'event', values='device_id').reset_index()

In [52]:
df_1

event,utm_source,app_install,app_start,choose_item,purchase,register,search,tap_basket
0,-,32460,52268,41276,16598,16527,50282,32076
1,facebook_ads,13916,25957,20687,9017,8672,24963,16362
2,google_ads,26286,38092,29495,11339,11405,36495,22588
3,instagram_ads,20096,31045,24503,10762,10750,29805,19339
4,referal,9282,15924,12849,6362,6193,15358,10395
5,vk_ads,23189,34079,27001,12364,12421,32713,21440
6,yandex-direct,29368,40707,31501,12028,12342,39018,24110


In [56]:
df_1['cr']=round(df_1.purchase/df_1.app_start*100, 1)
df_1.sort_values('cr',ascending=False)

event,utm_source,app_install,app_start,choose_item,purchase,register,search,tap_basket,cr
4,referal,9282,15924,12849,6362,6193,15358,10395,40.0
5,vk_ads,23189,34079,27001,12364,12421,32713,21440,36.3
1,facebook_ads,13916,25957,20687,9017,8672,24963,16362,34.7
3,instagram_ads,20096,31045,24503,10762,10750,29805,19339,34.7
0,-,32460,52268,41276,16598,16527,50282,32076,31.8
2,google_ads,26286,38092,29495,11339,11405,36495,22588,29.8
6,yandex-direct,29368,40707,31501,12028,12342,39018,24110,29.5


Самую низкую конверсию в покупку показывают пользователи, пришедшие с яндекс-директ

#### 9. Пользователи, пришедшие с какого платного канала, имеют медианный первый чек выше? 
(учитываются только первые покупки пользователей)

In [9]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [58]:
purchases.groupby('utm_source', as_index=False).agg({'purchase_sum':'median'}).sort_values('purchase_sum', ascending=False)

,utm_source,purchase_sum
0,-,398.5
4,referal,395.5
3,instagram_ads,393.5
5,vk_ads,393.0
6,yandex-direct,392.5
2,google_ads,390.5
1,facebook_ads,389.0


Таким образом, с канала "Реферальная программа" приходят пользователи, у которых медианный первый чек выше

#### 10. Посчитать какой платный канал привлечения имеет самый высокий ROMI

Данные по затратам на рекламу:\
Яндекс – 10 491 707 руб.\
Гугл – 10 534 878 руб.\
Фейсбук – 8 590 498 руб.\
Инстаграм – 8 561626 руб.\
ВК – 9 553 531руб. \
ROMI - отдача на инвестиции

In [61]:
romi = df.groupby('utm_source', as_index=False).agg({'purchase_sum':'sum'}) #считаем сначала выручку

In [62]:
romi

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [63]:
def get_romi(romi, yndx, ggl, fb, ig, vk):
    
    
    yndx_income = romi.query('utm_source=="yandex-direct"').purchase_sum.item()
    ggl_income = romi.query('utm_source=="google_ads"').purchase_sum.item()
    fb_income = romi.query('utm_source=="facebook_ads"').purchase_sum.item()
    ig_income = romi.query('utm_source=="instagram_ads"').purchase_sum.item()
    vk_income = romi.query('utm_source=="vk_ads"').purchase_sum.item()
    
    yndx_romi = round((yndx_income-yndx)/yndx*100, 2)
    ggl_romi = round((ggl_income-ggl)/ggl*100, 2)
    fb_romi = round((fb_income-fb)/fb*100, 2)
    ig_romi = round((ig_income-ig)/ig*100, 2)
    vk_romi = round((vk_income-vk)/vk*100, 2)
    romi_dict = {'yandex romi': yndx_romi, 'google romi': ggl_romi, 'fb romi': fb_romi, 'ig romi': ig_romi, 'vk_romi': vk_romi}
    
    return romi_dict

In [64]:
get_romi(romi, 10491707, 10534878, 8590498, 8561626, 9553531)

{'yandex romi': 32.63,
 'google romi': 22.15,
 'fb romi': 42.6,
 'ig romi': 69.91,
 'vk_romi': 71.56}

In [68]:
#более легкий способ
print("ROMI Facebook:", round((12249901/8590498 -1)*100, 2))
print("ROMI Google:", round((12868276/10534878 -1)*100, 2))
print("ROMI Яндекс:", round((13915368/10491707 -1)*100, 2))
print("ROMI ВК:", round((16389652/9553531 -1)*100, 2))
print("ROMI Instagram:", round((14546969/8561626 -1)*100, 2))

ROMI Facebook: 42.6
ROMI Google: 22.15
ROMI Яндекс: 32.63
ROMI ВК: 71.56
ROMI Instagram: 69.91


У ВК самый высокий ROMI, то есть самый оправданный канал маркетинга. На каждый вложенный рубль - отдача 71 р.